In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq
import os

In [2]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm = ChatGroq(model_name="qwen-2.5-32b")

In [3]:
messages=[
    SystemMessage("You are a translator your task is convert English to Bangla language."),
    HumanMessage("Assalamualikum, how are you bahi!")
]

In [4]:
llm.invoke(messages)

AIMessage(content='আসসালামুযালাইকুম, আপনি কেমন আছেন ভাই!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 39, 'total_tokens': 79, 'completion_time': 0.101349382, 'prompt_time': 0.004357315, 'queue_time': 0.238966439, 'total_time': 0.105706697}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d889268-89c8-49ef-b461-f5e976836bd2-0', usage_metadata={'input_tokens': 39, 'output_tokens': 40, 'total_tokens': 79})

In [5]:
response=llm.invoke(messages)

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(response)

'আসসালামুযালাইকুম, তুমি কেমন আছো ভাই!'

In [7]:
## using LCEL careate chain
chain=llm|parser
chain.invoke(messages)

'আসসালামুযালাইকুম, তুমি কেমন আছো বাহি!'

## Now create all things more efficient way

In [26]:
from langchain_core.prompts import ChatPromptTemplate
generic_prompt="You are a Translator. Translate all data so your task is convert English to {language} language."
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',generic_prompt),
        ("user", "{user_input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['language', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a Translator. Translate all data so your task is convert English to {language} language.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])

In [13]:
prompt.invoke(
    {"language":"Bangla",'user_input': "I'm good, how was your day?"}
    
)

ChatPromptValue(messages=[SystemMessage(content='You are a Translator and your task is convert English to Bangla language.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm good, how was your day?", additional_kwargs={}, response_metadata={})])

In [15]:
result = prompt.invoke(
    {"language": 'Bangla', "user_input":"I'm good, how about you!"}
)

In [16]:
result.to_messages()

[SystemMessage(content='You are a Translator and your task is convert English to Bangla language.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm good, how about you!", additional_kwargs={}, response_metadata={})]

In [27]:
chain = prompt|llm|parser

In [18]:
chain.invoke(
    {"language": 'Bangla', "user_input":"I'm good, how about you!"}
)

'আমি ভালো আছি, তুমি কেমন?'

In [23]:

text = "You are a friendly and patient math tutor. You will help students learn algebra by providing clear explanations and step-by-step solutions. You will offer hints when needed and identify errors in their work. You will encourage them to keep trying.\n\n\n\nSolve the equation: 2x + 4 = x - 3.\n\n\n\nTo solve this equation, we\u2019ll isolate x by first subtracting x from both sides. This gives x + 4 = -3. Do you understand this step?\n\n\n\nYes.\n\n\n\nNext, subtract 4 from both sides: x = -3 - 4, which simplifies to x = -7. Do you understand this step?\n\n\n\nYes.\n\n\n\nGreat! The solution to the equation 2x + 4 = x - 3 is x = -7. Would you like to try another equation?\n"
  

In [28]:
response = chain.invoke(
    {"language": 'Bangla', "user_input":text}
)
print(response)

আপনি একজন মিষ্টি এবং দীর্ঘায়মান গণিত শিক্ষক। আপনি ছাত্রদের বোঝাতে পারবেন বীজগণিত এবং ধীরে ধীরে সমাধান প্রদান করে তাদের শেখাবেন। আপনি প্রয়োজনে সূচনা দিবেন এবং তাদের কাজে ত্রুটি চিহ্নিত করবেন। আপনি তাদের অবিরত চেষ্টা করার জন্য অনুপ্রেরণা দিবেন।



সমীকরণটি সমাধান করুন: 2x + 4 = x - 3.



এই সমীকরণটি সমাধান করতে, আমরা প্রথমে উভয় পক্ষ থেকে x বিয়োগ করে x কে আলাদা করব। এটি x + 4 = -3 দেয়। আপনি এই ধাপটি বুঝছেন?



হ্যাঁ।



আমরা পরের ধাপে উভয় পক্ষ থেকে 4 বিয়োগ করব: x = -3 - 4, যা সরলীকৃত হয় x = -7। আপনি এই ধাপটি বুঝছেন?



হ্যাঁ।



সুন্দর! সমীকরণ 2x + 4 = x - 3 এর সমাধান x = -7। আপনি আরেকটি সমীকরণ চেষ্টা করতে চান?
